# Trying Task-Agnostic Amortized Inference of Gaussian Process Hyperparameters for regression:




In [2]:
# first we make sure that we have everything:
! pip install easydict PyYAML tensorboardX amor-hyp-gp 

In [12]:
! pip install "torch==1.6.0"

In [1]:
# imports:
import numpy as np
import torch

from easydict import EasyDict as edict
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from scipy import stats

import matplotlib.pyplot as plt

# from the package:
from ahgp.inference.hyperparam import hyperparam
from ahgp.gp.gp_helper import cal_kern_spec_mix_nomu_sep, GP_noise, standardize
from ahgp.inference.predict import predict

ModuleNotFoundError: No module named 'torch'

In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('../AHGP/src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
import math
from ahgp.gp.gp_helper import cal_kern_spec_mix_nomu_sep, cal_marg_likelihood, standardize, cal_marg_likelihood_single_L, cal_kern_spec_mix_sep

In [2]:

filename = "./data/regression_datasets/housing.data"
# filename = "./data/regression_datasets/concrete.data"
model_config_filename = "./config/model.yaml"

In [3]:
rand_seed = 10
npr = np.random.RandomState(rand_seed)
data = np.loadtxt(filename)
data = shuffle(data, random_state=npr)
x, y = data[:, :-1], data[:, -1]
data_dim = x.shape[1]
# normalize x and y, AHGP will take in normalized x and y only

x_t, x_v, y_t, y_v = train_test_split(x, y, test_size=.1, random_state=npr)
num_data = x_t.shape[0]
# normalize x and y, AHGP will take in normalized x and y only
x_t, x_v, _, _ = standardize(x_t, x_v)
x_t = x_t*0.1
x_v = x_v*0.1
y_t, mean_y_train, std_y_train = standardize(y_t)


mu_test, var_test = predict(x_t,y_t,x_v,model_config_filename,use_gpu=False)

mu_test = mu_test * std_y_train + mean_y_train
var_test = var_test * std_y_train**2

rmse = np.mean((mu_test - y_v) ** 2) ** .5
log_likelihood = np.mean(np.log(stats.norm.pdf(
                          y_v,
                          loc=mu_test,
                          scale=var_test ** 0.5)))
print(rmse)
print(log_likelihood)

NameError: name 'shuffle' is not defined